# Домашнее задание 2

В этом задании:

1. Сделаем регрессию над данными через scikit-learn: сначала через регресию, потом через бустинг.
2. Сравним результаты с константным предсказанием.
3. Сделаем нейронную сеть на полносвязных слоях, обучим над теми же данными - и сравним с лин. регрессией и бустингом.



In [1]:
import random

import numpy as np
import pandas as pd

# Для воспроизводимости
seed = 0
np.random.seed(seed)
random.seed(seed)

In [2]:
df = pd.read_csv("insurance.csv")

### Задание №1:
Cделайте train/test split на данных в пропорции 0.2/0.8, залейте в лмс код, который в `df_train`, `df_test`
сохранит датафрейм с тренировочными и тестовыми данными соответственно.

P.S Использовать train_test_split из scikit-learn запрещено - разбивайте вручную через индексы.

In [3]:
idx_test = np.random.randint(0, df.shape[0], int(0.2 * df.shape[0]))
df_test = df.iloc[idx_test]
df_train = df.drop(index=idx_test)

### Задание №2:
Сделайте `OHE` на колонки sex, region, smoker.

Нужно сделать как на train, так и на test датасете.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.
При этом в `df_train`, `df_test` должны остаться старые колонки - т.е. его нужно обогатить.


In [4]:
from sklearn.preprocessing import OneHotEncoder

columns_to_encode = ['sex', 'region', 'smoker']
ohe = OneHotEncoder()

train_encoded = ohe.fit_transform(df_train[columns_to_encode])
test_encoded = ohe.transform(df_test[columns_to_encode])

encoded_columns = ohe.get_feature_names_out(columns_to_encode)

train_encoded_df = pd.DataFrame(train_encoded.toarray(), columns=encoded_columns, index=df_train.index)
test_encoded_df = pd.DataFrame(test_encoded.toarray(), columns=encoded_columns, index=df_test.index)

df_train = pd.concat([df_train, train_encoded_df], axis=1)
df_test = pd.concat([df_test, test_encoded_df], axis=1)

print(df_train.columns)
print(df_test.columns)

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges',
       'sex_female', 'sex_male', 'region_northeast', 'region_northwest',
       'region_southeast', 'region_southwest', 'smoker_no', 'smoker_yes'],
      dtype='object')
Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges',
       'sex_female', 'sex_male', 'region_northeast', 'region_northwest',
       'region_southeast', 'region_southwest', 'smoker_no', 'smoker_yes'],
      dtype='object')


### Задание №4:
Нормализуйте колонки, которые вы отметили в квизе.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.

Сдайте код, который модифицирует `df_train` и `df_test` так, чтобы численные колонки из прошлого пункта стали нормированы.

In [5]:
num_cols = ['age', 'bmi', 'charges']
df_train[num_cols]

,age,bmi,charges
0,19,27.900,16884.92400
1,18,33.770,1725.55230
2,28,33.000,4449.46200
3,33,22.705,21984.47061
4,32,28.880,3866.85520
...,...,...,...
1333,50,30.970,10600.54830
1334,18,31.920,2205.98080
1335,18,36.850,1629.83350
1336,21,25.800,2007.94500


In [6]:
train_mean = df_train[num_cols].mean()
train_std = df_train[num_cols].std()
df_train[num_cols] = (df_train[num_cols] - train_mean) / train_std
df_test[num_cols] = (df_test[num_cols] - train_mean) / train_std

In [7]:
y_train = df_train.pop("charges")
y_test = df_test.pop("charges")

### Задание №5
Реализуйте функцию, считающую `MSE` метрику.

Ваша функция должна уметь принимать torch.Tensor, numpy-массивы и pd.Series.

In [15]:
def metric(preds, y):
    return ((preds - y) ** 2).mean()

### Задание №6
Реализуйте бейзлайн на `LinearRegression` и `GradientBoostingRegressor`, отправьте метрики на _тестовой выборке_ в ЛМС.

Используйте гиперпараметры по-умолчанию в обоих моделях.

In [11]:
df_train

,age,sex,bmi,children,smoker,region,sex_female,sex_male,region_northeast,region_northwest,region_southeast,region_southwest,smoker_no,smoker_yes
0,-1.422596,female,-0.470495,0,yes,southwest,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,-1.493645,male,0.496994,1,no,southeast,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,-0.783156,male,0.370083,3,no,southeast,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,-0.427912,male,-1.326732,0,no,northwest,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,-0.498961,male,-0.308972,0,no,northwest,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,0.779918,male,0.035500,3,no,northwest,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1334,-1.493645,female,0.192078,0,no,northeast,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1335,-1.493645,female,1.004638,0,no,southeast,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1336,-1.280498,female,-0.816616,0,no,southwest,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

lr = LinearRegression()
lr.fit(df_train.drop(columns=['sex', 'region', 'smoker']), y_train)
print(metric(lr.predict(df_test.drop(columns=['sex', 'region', 'smoker'])), y_test))

gb = GradientBoostingRegressor()
gb.fit(df_train.drop(columns=['sex', 'region', 'smoker']), y_train)
print(metric(gb.predict(df_test.drop(columns=['sex', 'region', 'smoker'])), y_test))

# for model in (LinearRegression(), GradientBoostingRegressor()):
#     model.fit(df_train.drop(columns=['sex', 'region', 'smoker']), y_train)
#     preds = model.predict(df_test.drop(columns=['sex', 'region', 'smoker']))
#     score = metric(preds, y_test)
#     print(model, score)

0.3295121966372632
0.21326231867810302


### Задание №7
Вычислите среднее значение целевой переменной на тренировочной выборке (train).

Подсчитайте MSE при константном предсказании этим средним и отправьте его в ЛМС.

In [24]:
dummy_score = metric(y_train.mean(), y_test)
dummy_score

1.1045869096529353

### Задание №8
Создайте сеть, состоящую из одного слоя Linear, залейте в лмс код, который в `model` запишет вашу модель

В качестве признаков используйте все колонки в текущем датасете, за исключением таргета

In [27]:
from torch import nn


def build_model():
    return nn.Linear(in_features=11, out_features=1)


model = build_model()

In [29]:
import torch

torch.random.manual_seed(seed)
t_x_train = torch.from_numpy(df_train.drop(columns=['sex', 'region', 'smoker']).to_numpy().astype(float)).to(dtype=torch.float32)
t_y_train = torch.from_numpy(y_train.drop(columns=['sex', 'region', 'smoker']).to_numpy()).to(dtype=torch.float32)

### Задание №9
Напишите функцию `train_loop`, которая будет учить модель по данным на 2к итераций.
Считайте, что данные уже хранятся в переменных `t_x_train`, `t_y_train`.
Ваша функция `train_loop` должна вернуть список из лоссов на каждой итерации (т.е. список длины 2000).

Используйте `learning_rate=1e-2` в оптимизаторе.

Для простоты за одну итерацию делайте проход вперед и проход назад на всех наших обучающих данных.
Это будет полный градиентный спуск (не по батчам) - можем себе позволить, данных немного.

_Подсказка 1_: Вам не обязательно учить модель на видеокарте, CPU будет достаточно.

_Подсказка 2_: `tqdm` - это библиотека, которая рисует прогресс итераций

In [39]:
t_x_train.squeeze(1).shape

torch.Size([1097, 11])

In [30]:
from tqdm import tqdm
from torch.optim import Adam

def metric(preds, y):
    return ((preds - y) ** 2).mean()


def train_loop(model: nn.Module) -> list[float]:
    losses = []
    optimizer = Adam(model.parameters(), lr=1e-2)

    for _ in tqdm(range(2000)):
        optimizer.zero_grad()
        loss = metric(model(t_x_train), t_y_train)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    return losses

### Задание №10
Обучите модель, состоящую из одного слоя `Linear`.
Приложите в ЛМС метрику `MSE` на тестовых данных.
Используйте `learning_rate=1e-2` в оптимизаторе.

Когда будете тестировать, не забудьте перенести тестовые данные в `torch.Tensor`

In [44]:
t_x_test = torch.from_numpy(df_test.drop(columns=['sex', 'region', 'smoker']).to_numpy().astype(float)).to(dtype=torch.float32)
t_y_test = torch.from_numpy(y_test.to_numpy()).to(dtype=torch.float32)

train_loop(model)
with torch.no_grad():
    y_pred = model(t_x_test)[:, 0]
metric(y_pred, t_y_test)

100%|██████████| 2000/2000 [00:01<00:00, 1021.08it/s]


tensor(1.4592)

### Задание №11
Вам необходимо усложнить существующую нейронную сеть, добавив один скрытый слой. 

Используйте следующие параметры: 

Размерность скрытого слоя: 6 нейронов, функция активации -  `ReLU`

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [14]:
def build_model():
    hid_dim = 6
    return nn.Sequential(
        nn.Linear(in_features=11, out_features=hid_dim),
        nn.ReLU(),
        nn.Linear(hid_dim, 1),
    )


model = build_model()

### Задание №12
Приложите в ЛМС метрику качества этой сети после 2к итераций обучения.
Эту модель можно обучить на CPU, не обязательно на видеокарте.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

In [45]:
train_loop(model)
with torch.no_grad():
    y_pred = model(t_x_test)[:, 0]
metric(y_pred, t_y_test)

100%|██████████| 2000/2000 [00:01<00:00, 1007.24it/s]


tensor(1.4592)

### Задание №13
Добавьте дополнительные слои в нейронную сеть

Вам необходимо усложнить нейронную сеть, добавив еще 2-3 скрытых слоя с такими же размерностями, как в предыдущем задании.

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [16]:
def build_model():
    hid_dim = 6
    return nn.Sequential(
        nn.Linear(in_features=11, out_features=hid_dim),
        nn.ReLU(),
        nn.Linear(hid_dim, hid_dim),
        nn.ReLU(),
        nn.Linear(hid_dim, hid_dim),
        nn.ReLU(),
        nn.Linear(hid_dim, 1),
    )


model = build_model()

### Задание №14
Приложите в ЛМС метрику качества после 2к итераций обучения.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

Эту модель можно обучить на CPU, не обязательно на видеокарте.